# 简单示例
这里将通过一个简单示例，让读者留下一个初步的Alink印象。

您可以选择如下操作：
* [在Alink使用指南观看](https://www.yuque.com/pinshu/alink_guide/pyalink_tutorial_1_1)
* [在Google Colab运行](https://colab.research.google.com/github/xuyang1706/pyalink_tutorial/blob/main/pyalink_tutorial_1_1.ipynb)
* [在 GitHub 查看源代码](https://github.com/xuyang1706/pyalink_tutorial/blob/main/pyalink_tutorial_1_1.ipynb)
* [下载ipynb文件](https://github.com/xuyang1706/pyalink_tutorial/raw/main/pyalink_tutorial_1_1.ipynb)

## 检查Alink环境
判断是否已经安装了PyAlink，如果没有安装会自动触发安装。
在安装前会判断是否在中国，便于通过中国的镜像服务器加速软件下载过程。

In [ ]:
%%bash

if [ $(pip freeze | grep pyalink | wc -l) -gt 0 ]
then
    echo "PyAlink already installed."
else
    IP_LOCATION=$(curl http://ip-api.com/json/?fields=1)
    echo $IP_LOCATION
    echo "Installing PyAlink..."
    if [[ "$IP_LOCATION" == *"China"* ]]
    then
        echo "Use mirror in China"
        pip3 install pyalink -i https://mirrors.aliyun.com/pypi/simple/
    else
        echo "Use default mirror"
        pip3 install pyalink
    fi
    echo "PyAlink installed."
fi

## 导入PyAlink
设置本地运行环境，设置本地数据文件路径（默认为当前执行路径）

In [ ]:
from pyalink.alink import *
useLocalEnv(1)

import os

DATA_DIR = "." + os.sep

## 读取网络文件与显示
我们以常用的鸢尾花（iris）数据集为例，演示一下如何读取数据。该数据可以通过网络链接直接下载。Alink的CSV格式数据源读取组件，不但可以读取本地文件，还可以直接读取网络文件；在该组件读取文件的时候需要指定各列数据的名称和类型。下面是具体的代码。在此，读取出数据后，选择前5条数据进行打印输出

In [ ]:
source = CsvSourceBatchOp()\
    .setFilePath("http://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data")\
    .setSchemaStr("sepal_length double, sepal_width double, petal_length double, "
                  + "petal_width double, category string")

source.firstN(5).print()

## 数据采样及写出
下一步，我们对数据进行采样，采样数为10条，然后连接到CSV格式输出组件CsvSinkBatchOp，并设置相应的输出路径。可以设置覆盖写参数为True。

In [ ]:
source.sampleWithSize(10)\
    .link(
        CsvSinkBatchOp()\
            .setFilePath(DATA_DIR + "iris_10.csv")\
            .setOverwriteSink(True)
    )

BatchOperator.execute()

## 读取本地数据
读取刚才写出的CSV文件，使用LazyPrint方式打印输出，并使用GroupBy操作，得到各类别的样本个数。

In [ ]:
CsvSourceBatchOp()\
    .setFilePath(DATA_DIR + "iris_10.csv")\
    .setSchemaStr("sepal_length double, sepal_width double, petal_length double, "
                  + "petal_width double, category string")\
    .lazyPrint(-1, "【 采样数据 】")\
    .groupBy("category", "category, COUNT(*) AS cnt")\
    .print()